In [1393]:
p = np.random.randint( 3, 10 )
q = np.random.randint( 3, 10 )
print( "PQ", p, q)

# construct a random matrix of size m, n
A = np.random.randn( p, q )


def downdateEx( A ):

    p = A.shape[ 0 ]
    q = A.shape[ 1 ]
    r = min( p, q )

    D = A[ :, -1 ]; D = D.reshape( p, 1 )
    
    # retrieve a first SVD representation
    U, s, Vt = np.linalg.svd( A, full_matrices=False )

    Q1, R1 = np.linalg.qr( 
        np.dot(
            np.dot( U, U.T ) - np.identity( p ),
            D
        )
    )

    Q2, R2 = np.linalg.qr(
        np.dot(
            np.identity( q ) - np.dot( Vt.T, Vt ),
            np.vstack(
                (np.zeros( (q-D.shape[1], D.shape[1] ) ),
                np.identity( D.shape[1] ))
            )
        )
    )

    l = D.shape[ 1 ]

    Q1p = Q1[ :, :l ]
    Q2p = Q2[ :, :l ]

    R1p = R1[ :l, : ]
    R2p = R2[ :l, : ]


    K = np.dot(

        np.vstack((
            -np.dot( U.T, D ),
            R1p
        )),

        np.hstack((
            np.dot(
                np.hstack((
                    np.zeros(( l, q-l )),
                    np.identity( l )
                )),
                Vt.T
            ),
            np.transpose( R2p )
        ))
    )

    K = K + np.diag( np.append( s, 0 ) )

    Uk, sk, Vtk = np.linalg.svd( K, full_matrices=False )


    Ud = np.dot(
        np.hstack((
            U,
            Q1p
        )),
        Uk
    )

    sd = sk
    sd[ abs( sd ) < 1.0e-10 ] = 0

    Vd = np.dot(
        np.hstack((
            Vt.T, Q2p
        )),
        Vtk.T
    )


    Vd = Vd[ :q-l, : ]
    Vdt = np.transpose( Vd )

    return Ud, sd, Vdt
    
    
Ud, sd, Vdt = downdateEx( A )
    
    
print ( Ud.shape, sd.shape, Vdt.shape )
nm = Ud.shape[ 0 ]
nn = Vdt.shape[ 1 ]
nr = min( nm, nn )

Ud = Ud[ :, :nr ]
sd = sd[ :nr ]
Vdt = Vdt[ :nr, : ]
print ( Ud.shape, sd.shape, Vdt.shape )
    

Ai = np.dot( Ud, np.dot( np.diag(sd), Vdt ))
Am = A[ :, :-1 ]


print( "Matrix", np.allclose(  Am, Ai  ) )


sx = sd.copy()
nzindx = sx > 0; sx[ nzindx ] = 1.0/sx[ nzindx ]

print( "InvI", np.dot( np.dot( np.transpose(Vdt), np.dot( np.diag( sx ), np.transpose( Ud ) ) ), Am ) )
print( "Error: ", np.linalg.norm(  np.dot( Am, np.dot( 
    np.dot( np.transpose(Vdt), np.dot( np.diag( sx ), np.transpose( Ud ) ) ), 
    Am ) ) - Am ) )



def svdsolve(Ainp):
    u, s, v = np.linalg.svd(Ainp, full_matrices=False)
    Ainvo = np.dot (v.T, np.dot(np.diag(s**-1),u.T))
    return(Ainvo)

print( "INvC", np.dot( svdsolve(Am), Am ) )
print( "Error: ", np.linalg.norm(  np.dot( Am, np.dot( svdsolve(Am), Am ) ) - Am ) )



PQ 5 8
(5, 6) (6,) (6, 7)
(5, 5) (5,) (5, 7)
Matrix True
InvI [[ 0.94447615  0.03299484  0.16526108  0.05391296 -0.10293845  0.06804356
  -0.0768649 ]
 [ 0.03299484  0.97521251 -0.09759138 -0.04517121  0.02876056 -0.10002837
   0.02621168]
 [ 0.16526108 -0.09759138  0.50804439 -0.15890876  0.31022967 -0.19545685
   0.23108908]
 [ 0.05391296 -0.04517121 -0.15890876  0.91435427  0.0177842  -0.21715391
   0.02528638]
 [-0.10293845  0.02876056  0.31022967  0.0177842   0.60638972 -0.24668588
  -0.2642822 ]
 [ 0.06804356 -0.10002837 -0.19545685 -0.21715391 -0.24668588  0.2310695
  -0.12972147]
 [-0.0768649   0.02621168  0.23108908  0.02528638 -0.2642822  -0.12972147
   0.82045348]]
Error:  5.559713776130884e-15
INvC [[ 0.94447615  0.03299484  0.16526108  0.05391296 -0.10293845  0.06804356
  -0.0768649 ]
 [ 0.03299484  0.97521251 -0.09759138 -0.04517121  0.02876056 -0.10002837
   0.02621168]
 [ 0.16526108 -0.09759138  0.50804439 -0.15890876  0.31022967 -0.19545685
   0.23108908]
 [ 0.05391296

In [29]:
import numpy as np
import numpy.random as rnd
import time

x = np.arange(1,10, 1).reshape((3,3))*0.1
y = np.diag(x)
print( np.diag( y**-1 ) )
print( np.matrix(np.diag(y)).I )

[[10.          0.          0.        ]
 [ 0.          2.          0.        ]
 [ 0.          0.          1.11111111]]
[[10.          0.          0.        ]
 [ 0.          2.          0.        ]
 [ 0.          0.          1.11111111]]
